### 000_spn_warehouse_ownership_transfer

#### Inventory and transfer ownership of Microsoft Fabric Warehouses via REST API.

This notebook provides functions to:
1. List all warehouses in the current workspace (by default) with metadata
2. Transfer warehouse ownership to the authenticated Service Principal

Why transfer ownership to an SPN?
- User-owned warehouses break when the owner leaves (account deactivated)
- User-owned warehouses break if owner doesn't sign in for 30 days (token expiry)
- SPN ownership eliminates human dependency and enables automated token refresh

Prerequisites:
- Service Principal with Fabric API permissions
- Tenant setting "Service principals can use Fabric APIs" enabled
- SPN added to allowed security group in tenant settings
- SPN has Contributor, Member, or Admin role on target workspace

IMPORTANT: The takeover API transfers ownership to the CALLING IDENTITY.
When this notebook runs authenticated as the SPN, the SPN becomes the new owner.

In [ ]:
import requests
import notebookutils.credentials as cred
from typing import List, Optional, Dict, Any

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================
# Choose ONE authentication method: Key Vault (recommended) OR Manual

# -----------------------------------------------------------------------------
# Option A: Key Vault-based credentials (RECOMMENDED for production)
# -----------------------------------------------------------------------------
# Uncomment and configure the following block to use Key Vault:
#
# 
#
# KEY_VAULT_NAME = "your-keyvault-name"  # e.g., "kv-fabric-prod"
# KEY_VAULT_URL = f"https://{KEY_VAULT_NAME}.vault.azure.net/"
#
# TENANT_ID = cred.getSecret(KEY_VAULT_URL, "aad-tenant-id")
# SP_CLIENT_ID = cred.getSecret(KEY_VAULT_URL, "fabric-spn-client-id")
# SP_CLIENT_SECRET = cred.getSecret(KEY_VAULT_URL, "fabric-spn-client-secret")

# -----------------------------------------------------------------------------
# Option B: Manual credentials (for development/testing ONLY)
# -----------------------------------------------------------------------------
# WARNING: Never commit credentials to source control!
TENANT_ID = "" #"tenantid"
SP_CLIENT_ID = "" #"spclientid"
SP_CLIENT_SECRET = "" #"spsecret"

# -----------------------------------------------------------------------------
# Workspace Configuration
# -----------------------------------------------------------------------------
# Set to None to use current workspace (auto-detected from runtime context)
# Set to a specific GUID string to target a different workspace
WORKSPACE_ID_OVERRIDE: Optional[str] = None

def get_workspace_id() -> str:
    """
    Get the target workspace ID.
    
    Uses WORKSPACE_ID_OVERRIDE if set, otherwise auto-detects from runtime context.
    
    Returns:
        str: Workspace ID (GUID)
    """
    if WORKSPACE_ID_OVERRIDE:
        return WORKSPACE_ID_OVERRIDE
    
    # Auto-detect from Fabric runtime context
    context = mssparkutils.runtime.context
    return context["currentWorkspaceId"]

# -----------------------------------------------------------------------------
# API Configuration
# -----------------------------------------------------------------------------
FABRIC_API_BASE_URL = "https://api.fabric.microsoft.com/v1"
POWERBI_API_BASE_URL = "https://api.powerbi.com/v1.0/myorg"
OAUTH_TOKEN_URL_TEMPLATE = "https://login.microsoftonline.com/{tenant_id}/oauth2/token"
FABRIC_RESOURCE_URI = "https://api.fabric.microsoft.com"

In [ ]:
# =============================================================================
# AUTHENTICATION
# =============================================================================
def get_access_token(
    tenant_id: str,
    client_id: str,
    client_secret: str
) -> str:
    """
    Acquire OAuth2 access token from Microsoft Entra ID using client credentials flow.
    
    This function authenticates the Service Principal against Microsoft's OAuth2
    token endpoint and returns a bearer token for Fabric API calls.
    
    Args:
        tenant_id: Azure AD tenant identifier (GUID)
        client_id: Service Principal application (client) ID
        client_secret: Service Principal client secret value
    
    Returns:
        str: Bearer access token for Fabric API authentication
    
    Raises:
        requests.HTTPError: If authentication fails (invalid credentials, 
                           insufficient permissions, etc.)
        KeyError: If response doesn't contain expected 'access_token' field
    
    Example:
        >>> token = get_access_token(TENANT_ID, SP_CLIENT_ID, SP_CLIENT_SECRET)
        >>> print(f"Token acquired: {token[:20]}...")
    """
    token_url = OAUTH_TOKEN_URL_TEMPLATE.format(tenant_id=tenant_id)
    
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    
    payload = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret,
        "resource": FABRIC_RESOURCE_URI
    }
    
    response = requests.post(token_url, data=payload, headers=headers)
    response.raise_for_status()
    
    token_data = response.json()
    access_token = token_data["access_token"]
    
    return access_token


# =============================================================================
# HELPER FUNCTIONS
# =============================================================================
def _get_auth_headers(access_token: str) -> dict:
    """
    Construct standard authorization headers for API requests.
    
    Args:
        access_token: Valid OAuth2 bearer token
    
    Returns:
        dict: Headers dictionary with Authorization and Content-Type
    """
    return {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

def _get_workspace_name(access_token: str, workspace_id: str) -> str:
    """
    Fetch workspace display name by ID.
    
    Args:
        access_token: OAuth2 bearer token
        workspace_id: Workspace GUID
    
    Returns:
        str: Workspace display name, or empty string if not found
    """
    url = f"{FABRIC_API_BASE_URL}/workspaces/{workspace_id}"
    headers = _get_auth_headers(access_token)
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json().get("displayName", "")
    except requests.HTTPError:
        return ""


# =============================================================================
# CORE FUNCTIONS - WAREHOUSE INVENTORY
# =============================================================================
def list_warehouses_admin(
    access_token: str,
    workspace_id: Optional[str] = None
) -> List[Dict[str, Any]]:
    """
    Retrieve all warehouses in a workspace using the Admin API (includes owner information).
    
    Uses /v1/admin/items endpoint which returns creatorPrincipal with owner details.
    Filters to Warehouse type in the specified workspace.
    
    IMPORTANT: Requires additional tenant setting:
    "Service principals can access read-only admin APIs" under Admin API settings.
    
    Args:
        access_token: OAuth2 bearer token from Entra ID
        workspace_id: Target workspace ID. If None, uses get_workspace_id() (current workspace).
    
    Returns:
        List[Dict]: List of warehouse objects with keys:
            - name: Warehouse display name
            - id: Warehouse GUID
            - type: "Warehouse"
            - workspace_id: Parent workspace GUID
            - owner_display_name: Owner's display name
            - owner_upn: Owner's user principal name (email)
    
    Raises:
        requests.HTTPError: If API call fails (check tenant settings if 403)
    
    Example:
        >>> warehouses = list_warehouses_admin(token)
        >>> for wh in warehouses:
        ...     print(f"{wh['name']} owned by {wh['owner_display_name']}")
    """
    if workspace_id is None:
        workspace_id = get_workspace_id()
    
    url = f"{FABRIC_API_BASE_URL}/admin/items"
    headers = _get_auth_headers(access_token)
    
    all_items = []
    continuation_token = None
    
    # Handle pagination
    while True:
        params = {
            "type": "Warehouse",
            "workspaceId": workspace_id
        }
        if continuation_token:
            params["continuationToken"] = continuation_token
        
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        
        data = response.json()
        
        # Admin API returns itemEntities, not value
        for entity in data.get("itemEntities", []):
            creator = entity.get("creatorPrincipal") or {}
            user_details = creator.get("userDetails") or {}
            
            all_items.append({
                "name": entity.get("name"),
                "id": entity.get("id"),
                "type": entity.get("type"),
                "workspace_id": entity.get("workspaceId"),
                "owner_display_name": creator.get("displayName"),
                "owner_upn": user_details.get("userPrincipalName")
            })
        
        continuation_token = data.get("continuationToken")
        if not continuation_token:
            break
    
    return all_items


def get_warehouse_inventory_dataframe(
    access_token: str,
    workspace_id: Optional[str] = None
):
    """
    Get warehouse inventory as a Spark DataFrame for display.
    
    Uses the Admin API to retrieve owner information.
    
    IMPORTANT: Requires tenant setting "Service principals can access read-only admin APIs"
    enabled under Admin API settings with the SPN's security group added.
    
    Args:
        access_token: OAuth2 bearer token from Entra ID
        workspace_id: Target workspace ID. If None, uses get_workspace_id() (current workspace).
    
    Returns:
        DataFrame: Spark DataFrame with columns:
            - workspace_name
            - workspace_id
            - warehouse_name
            - warehouse_id
            - owner_upn
            - owner_display_name
    
    Example:
        >>> df = get_warehouse_inventory_dataframe(token)
        >>> display(df)
    """
    if workspace_id is None:
        workspace_id = get_workspace_id()
    
    # Fetch workspace name
    workspace_name = _get_workspace_name(access_token, workspace_id)
    
    warehouses = list_warehouses_admin(access_token, workspace_id)
    
    # Build rows for DataFrame
    rows = []
    for wh in warehouses:
        rows.append({
            "workspace_name": workspace_name,
            "workspace_id": workspace_id,
            "warehouse_name": wh.get("name", ""),
            "warehouse_id": wh.get("id", ""),
            "owner_upn": wh.get("owner_upn", ""),
            "owner_display_name": wh.get("owner_display_name", "")
        })
    
    # Create Spark DataFrame with explicit column order
    if rows:
        df_warehouses = spark.createDataFrame(rows)
        # Ensure column order
        df_warehouses = df_warehouses.select(
            "workspace_name",
            "workspace_id", 
            "warehouse_name",
            "warehouse_id",
            "owner_upn",
            "owner_display_name"
        )
    else:
        # Empty DataFrame with schema
        from pyspark.sql.types import StructType, StructField, StringType
        schema = StructType([
            StructField("workspace_name", StringType(), True),
            StructField("workspace_id", StringType(), True),
            StructField("warehouse_name", StringType(), True),
            StructField("warehouse_id", StringType(), True),
            StructField("owner_upn", StringType(), True),
            StructField("owner_display_name", StringType(), True)
        ])
        df_warehouses = spark.createDataFrame([], schema)
    
    return df_warehouses


# =============================================================================
# CORE FUNCTIONS - WAREHOUSE OWNERSHIP TAKEOVER
# =============================================================================
def takeover_warehouse(
    access_token: str,
    warehouse_id: str,
    workspace_id: Optional[str] = None
) -> bool:
    """
    Transfer ownership of a single warehouse to the calling identity (SPN).
    
    The takeover API assigns ownership to the authenticated caller.
    When authenticated as an SPN, the SPN becomes the new owner.
    
    IMPORTANT: This uses the Power BI API endpoint, not the Fabric API.
    The token obtained for api.fabric.microsoft.com works for both.
    
    Args:
        access_token: OAuth2 bearer token from Entra ID
        warehouse_id: Target warehouse GUID
        workspace_id: Target workspace ID. If None, uses get_workspace_id()
    
    Returns:
        bool: True if takeover succeeded, False otherwise
    
    Raises:
        requests.HTTPError: If API call fails (permissions, invalid IDs, etc.)
    
    Example:
        >>> success = takeover_warehouse(token, "abc123-def456-...")
        >>> if success:
        ...     print("Ownership transferred to SPN")
    """
    if workspace_id is None:
        workspace_id = get_workspace_id()
    
    # Note: Takeover uses Power BI API endpoint, not Fabric API
    url = f"{POWERBI_API_BASE_URL}/groups/{workspace_id}/datawarehouses/{warehouse_id}/takeover"
    headers = _get_auth_headers(access_token)
    
    response = requests.post(url, headers=headers, json={})
    
    # 200 = success
    if response.status_code == 200:
        print(f"[SUCCESS] Ownership transferred for warehouse {warehouse_id}")
        return True
    else:
        print(f"[FAILED] Takeover failed for {warehouse_id}: HTTP {response.status_code}")
        print(f"         Response: {response.text}")
        return False


def takeover_warehouses_batch(
    access_token: str,
    warehouse_ids: Optional[List[str]] = None,
    workspace_id: Optional[str] = None
) -> Dict[str, bool]:
    """
    Transfer ownership of multiple warehouses to the calling identity (SPN).
    
    If no warehouse_ids provided, takes over ALL warehouses in the specified workspace.
    
    Args:
        access_token: OAuth2 bearer token from Entra ID
        warehouse_ids: List of warehouse GUIDs to take over.
                      If None or empty, takes over ALL warehouses in workspace.
        workspace_id: Target workspace ID. If None, uses get_workspace_id().
                     Required when warehouse_ids is not provided.
    
    Returns:
        Dict[str, bool]: Map of warehouse_id -> success status
    
    Example:
        >>> # Take over specific warehouses
        >>> results = takeover_warehouses_batch(token, ["id1", "id2"])
        
        >>> # Take over ALL warehouses in current workspace
        >>> results = takeover_warehouses_batch(token)
        >>> 
        >>> for wh_id, success in results.items():
        ...     status = "OK" if success else "FAILED"
        ...     print(f"{wh_id}: {status}")
    """
    if workspace_id is None:
        workspace_id = get_workspace_id()
    
    # If no specific IDs provided, get all warehouses in the workspace
    if not warehouse_ids:
        print("No warehouse IDs specified - retrieving all warehouses in workspace...")
        warehouses = list_warehouses_admin(access_token, workspace_id)
        warehouse_ids = [wh["id"] for wh in warehouses]
        
        # Show what we found with owner info
        for wh in warehouses:
            print(f"  Found: {wh['name']} (owner: {wh.get('owner_display_name', 'Unknown')})")
        
        print(f"Total: {len(warehouse_ids)} warehouse(s) to process.")
    
    if not warehouse_ids:
        print("[INFO] No warehouses found in workspace.")
        return {}
    
    # Process each warehouse
    results = {}
    for wh_id in warehouse_ids:
        try:
            success = takeover_warehouse(access_token, wh_id, workspace_id)
            results[wh_id] = success
        except requests.HTTPError as e:
            print(f"[ERROR] Exception during takeover for {wh_id}: {e}")
            results[wh_id] = False
    
    # Summary
    succeeded = sum(1 for v in results.values() if v)
    failed = len(results) - succeeded
    print(f"\n[SUMMARY] Processed {len(results)} warehouse(s): {succeeded} succeeded, {failed} failed")
    
    return results

In [ ]:
# Acquire access token (required before any API calls)
access_token = get_access_token(TENANT_ID, SP_CLIENT_ID, SP_CLIENT_SECRET)
print("Access token acquired successfully.")
print(f"Target workspace: {get_workspace_id()}\n")

In [ ]:
# =============================================================================
# STEP 1: INVENTORY WAREHOUSES
# =============================================================================
# Uncomment to view all warehouses in the workspace
df_inventory = get_warehouse_inventory_dataframe(access_token)
display(df_inventory)


# =============================================================================
# STEP 2: TRANSFER OWNERSHIP TO SPN
# =============================================================================
# Option A: Take over ALL warehouses in the current workspace
# results = takeover_warehouses_batch(access_token)

# Option B: Take over specific warehouses by ID
# warehouse_ids_to_takeover = [
#     "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx",
#     "yyyyyyyy-yyyy-yyyy-yyyy-yyyyyyyyyyyy",
# ]
# results = takeover_warehouses_batch(access_token, warehouse_ids=warehouse_ids_to_takeover)